In [ ]:
function normalizeTime(loop) {
    const start_time = 
}

In [ ]:
//TCP server to OF

const net = require('net');
const tcp_port = 11999;
const tcp_host = '127.0.0.1';

const server = net.createServer();
server.listen(tcp_port, tcp_host, () => {
    console.log('TCP Server is running on port ' + tcp_port + '.');
});

let sockets = [];

server.on('connection', function(sock) {
    console.log('CONNECTED: ' + sock.remoteAddress + ':' + sock.remotePort);
    sockets.push(sock);

    sock.on('data', function(data) {
        console.log('DATA ' + sock.remoteAddress + ': ' + data);
        // Write the data back to all the connected, the client will receive it as data from the server
        sockets.forEach(function(sock, index, array) {
            sock.write(sock.remoteAddress + ':' + sock.remotePort + " said " + data + '\n');
        });
    });

    // Add a 'close' event handler to this instance of socket
    sock.on('close', function(data) {
        let index = sockets.findIndex(function(o) {
            return o.remoteAddress === sock.remoteAddress && o.remotePort === sock.remotePort;
        })
        if (index !== -1) sockets.splice(index, 1);
        console.log('CLOSED: ' + sock.remoteAddress + ' ' + sock.remotePort);
    });
});

Server {
  _events: [Object: null prototype] {
    listening: [Function: bound onceWrapper] { listener: [Function (anonymous)] },
    connection: [Function (anonymous)]
  },
  _eventsCount: 2,
  _maxListeners: undefined,
  _connections: 0,
  _handle: null,
  _usingWorkers: false,
  _workers: [],
  _unref: false,
  allowHalfOpen: false,
  pauseOnConnect: false,
  [Symbol(kCapture)]: false,
  [Symbol(async_id_symbol)]: -1
}

TCP Server is running on port 11999.


In [ ]:
//OSC Server to touchOSC

const osc = require('osc');

const TCP_OSC_PORT = 7071;
const TCP_OSC_HOST = '192.168.1.39';

let tcp_osc_port = new osc.TCPSocketPort({});
// change to remote host/port (address of phone/tablet) - set touchOSC on device to TCPserver
tcp_osc_port.open(TCP_OSC_HOST, TCP_OSC_PORT); 

tcp_osc_port.on('ready', () => {
  console.log('ready TCP');
});


const UDP_OSC_PORT = 7071;
const UDP_OSC_HOST = '0.0.0.0';

// Bind to a UDP socket to listen for incoming OSC events.
let udp_osc_port = new osc.UDPPort({
    localAddress: UDP_OSC_HOST,
    localPort: UDP_OSC_PORT,
    remoteAddress: '127.0.0.1',
    remotePort: 7072
});

udp_osc_port.on("ready", () => {
    console.log('ready UDP');
});

udp_osc_port.open();

let osc_port = udp_osc_port;

const rawLoops = {};
const processedLoops = {};
let activeLoop = 'firstLoop';
let isRecording = false;
let isTouching = false;
const handlers = {};

function processLoop(loopKey) {
    processedLoops[loopKey] = rawLoops[loopKey];
}

handlers['/recording'] = (isRec) => {
    isRecording = isRec;
    if(isRecording) {
        rawLoops[activeLoop] = [{ts: Date.now(), pos: 'start'}];
    } else {
        rawLoops[activeLoop].push({ts: Date.now(), pos: 'end'});
        processLoop(activeLoop);
        sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))
    }
}
const f = float => ({type: 'f', value: float}); //helper for formatting floats for OSC
handlers['/pos'] = (x, y) => {
    if(isRecording) {
        rawLoops[activeLoop].push({ts: Date.now(), pos: {x, y}});
    }
    udp_osc_port.sendMsg({addres: '/touchPos', args: [f(x), f(y)]});
}
handlers['/pos/touch'] = (touching) => {
    isTouching = touching;
    udp_osc_port.sendMsg({addres: '/touching', args: [f(isTouching)]});
}

osc_port.on('message', (message) => {
    let {address, args} = message;
    console.log(message)
    if(handlers[address]) handlers[address](...args); 
});

EventEmitter {
  options: {
    localAddress: '0.0.0.0',
    localPort: 7071,
    remoteAddress: '127.0.0.1',
    remotePort: 7072
  },
  _events: [Object: null prototype] {
    data: [Function: bound ],
    open: [Function: bound ],
    ready: [Function (anonymous)],
    message: [Function (anonymous)]
  },
  _eventsCount: 4,
  socket: Socket {
    _events: [Object: null prototype] {
      error: [Array],
      listening: [Function: onListening]
    },
    _eventsCount: 2,
    _maxListeners: undefined,
    type: 'udp4',
    [Symbol(kCapture)]: false,
    [Symbol(async_id_symbol)]: 23,
    [Symbol(state symbol)]: {
      handle: [UDP],
      receiving: false,
      bindState: 1,
      connectState: 0,
      queue: undefined,
      reuseAddr: undefined,
      ipv6Only: undefined,
      recvBufferSize: undefined,
      sendBufferSize: undefined
    }
  }
}

ready UDP
CONNECTED: 127.0.0.1:63734
{ address: '/recording', args: [ 1 ] }
{ address: '/pos/touch', args: [ true ] }


TypeError: udp_osc_port.sendMsg is not a function
    at Object.handlers./pos/touch (evalmachine.<anonymous>:66:18)
    at osc.UDPPort.<anonymous> (evalmachine.<anonymous>:72:44)
    at osc.UDPPort.emit (node:events:390:28)
    at Object.osc.firePacketEvents (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/osc-transports.js:25:18)
    at osc.UDPPort.p.decodeOSC (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/osc-transports.js:85:17)
    at osc.UDPPort.emit (node:events:390:28)
    at Socket.<anonymous> (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/platforms/osc-node.js:102:18)
    at Socket.emit (node:events:390:28)
    at UDP.onMessage [as onmessage] (node:dgram:931:8)

{ address: '/pos', args: [ 0.4182790219783783, 0.7319419384002686 ] }


TypeError: udp_osc_port.sendMsg is not a function
    at Object.handlers./pos (evalmachine.<anonymous>:62:18)
    at osc.UDPPort.<anonymous> (evalmachine.<anonymous>:72:44)
    at osc.UDPPort.emit (node:events:390:28)
    at Object.osc.firePacketEvents (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/osc-transports.js:25:18)
    at osc.UDPPort.p.decodeOSC (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/osc-transports.js:85:17)
    at osc.UDPPort.emit (node:events:390:28)
    at Socket.<anonymous> (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/platforms/osc-node.js:102:18)
    at Socket.emit (node:events:390:28)
    at UDP.onMessage [as onmessage] (node:dgram:931:8)

...

{ address: '/pos/touch', args: [ false ] }


TypeError: udp_osc_port.sendMsg is not a function
    at Object.handlers./pos/touch (evalmachine.<anonymous>:66:18)
    at osc.UDPPort.<anonymous> (evalmachine.<anonymous>:72:44)
    at osc.UDPPort.emit (node:events:390:28)
    at Object.osc.firePacketEvents (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/osc-transports.js:25:18)
    at osc.UDPPort.p.decodeOSC (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/osc-transports.js:85:17)
    at osc.UDPPort.emit (node:events:390:28)
    at Socket.<anonymous> (/Users/avneeshsarwate/abletonAutomationExtraction/node_modules/osc/src/platforms/osc-node.js:102:18)
    at Socket.emit (node:events:390:28)
    at UDP.onMessage [as onmessage] (node:dgram:931:8)

{ address: '/recording', args: [ 0 ] }


Error: connect ETIMEDOUT 192.168.1.39:7071
    at TCPConnectWrap.afterConnect [as oncomplete] (node:net:1161:16)

CLOSED: 127.0.0.1 63734


In [7]:
processedLoops['firstLoop'][0]

{ ts: 1640469770824, pos: 'start' }

In [6]:
sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))

CLOSED: 127.0.0.1 49285
